In [1]:
import pandas as pd
DATA_DIR="data/"

# Read CSVs from the internet, and apply some initial cleaning.

In [2]:
import re
df_gics = pd.read_csv(DATA_DIR+"sp500-with-gics.csv") # Downloaded from Kaggle
df_ticker_marketcap = pd.read_csv(DATA_DIR+"sp500-marketcaps-062022.csv",) # Downloaded from https://www.liberatedstocktrader.com/sp-500-companies/
df_ticker_marketcap.columns=["Ticker","CompanyName","Weird Sector","MarketCap"] # Rename columns so they don't have invisible characters
df_ticker_marketcap["MarketCap"] = df_ticker_marketcap["MarketCap"].apply(lambda x: int(re.search(r"(\d+,)+\d+",x).group(0).replace(",",""))) # Transform MarketCap into int
df_ticker_marketcap

,Ticker,CompanyName,Weird Sector,MarketCap
0,AAPL,Apple Inc.,Electronic Technology,2199710059533
1,MSFT,Microsoft Corporation,Technology Services,1898477741492
2,GOOGL,Alphabet Inc.,Technology Services,1477080954208
3,GOOG,Alphabet Inc.,Technology Services,1470632746031
4,AMZN,"Amazon.com, Inc.",Retail Trade,1104025258743
...,...,...,...,...
499,IPGP,IPG Photonics Corporation,Electronic Technology,4783694257
500,PENN,"Penn National Gaming, Inc.",Consumer Services,4683822859
501,PVH,PVH Corp.,Consumer Non-Durables,4087941105
502,UAA,"Under Armour, Inc.",Consumer Non-Durables,3936047574


# Merge the two DataFrames
Since these two DFs were not pulled from the same source, they differ on the 500 indexed companies

In [3]:
# Find the common tickers
ticker_intersection = set.intersection(set(df_gics["Ticker"].values),set(df_ticker_marketcap["Ticker"].values))
print(len(ticker_intersection),"common sp500 tickers.")
# Look at tickers in gics csv, not in marketcap csv
gics_set_unique_tickers = set(df_gics["Ticker"].values).difference(ticker_intersection) 
print("sp500-gics.csv onlt companies:")
print([companyName for companyName in df_gics[df_gics.Ticker.isin(gics_set_unique_tickers)]["CompanyName"]])
# Look at tickers in merketcap csv, not in gics csv
marketcap_set_unique_tickers = set(df_ticker_marketcap["Ticker"].values).difference(ticker_intersection) 
print("sp500-marketcaps-062022.csv only companies:")
print([companyName for companyName in df_ticker_marketcap[df_ticker_marketcap.Ticker.isin(marketcap_set_unique_tickers)]["CompanyName"]])
# Restrict to common set of tickers
df_gics = df_gics[df_gics["Ticker"].isin(ticker_intersection)]
df_ticker_marketcap = df_ticker_marketcap[df_ticker_marketcap.Ticker.isin(ticker_intersection)]

474 common sp500 tickers.
sp500-gics.csv onlt companies:
['Alexion Pharmaceuticals', 'Ball Corp', 'Cabot Oil & Gas', 'Cerner', 'Concho Resources', 'Discovery, Inc. (Class A)', 'Discovery, Inc. (Class C)', 'Facebook, Inc.', 'FLIR Systems', 'Flowserve Corporation', 'Gap Inc.', 'Hanesbrands Inc', 'HollyFrontier Corp', 'IHS Markit Ltd.', 'Kansas City Southern', 'L Brands Inc.', 'Leggett & Platt', 'Maxim Integrated Products Inc', 'NOV Inc.', "People's United Financial", 'Perrigo', 'SL Green Realty', 'TechnipFMC', 'Unum Group', 'Varian Medical Systems', 'ViacomCBS', 'Vontier', 'Western Union Co', 'Willis Towers Watson', 'Xerox', 'Xilinx']
sp500-marketcaps-062022.csv only companies:
['Meta Platforms, Inc.', 'Moderna, Inc.', 'NXP Semiconductors N.V.', 'Warner Bros. Discovery, Inc. – Series A', 'VICI Properties Inc.', 'Coterra Energy Inc.', 'Willis Towers Watson Public Limited Company', 'Ball Corporation', 'Match Group, Inc.', 'Constellation Energy Corporation', 'Monolithic Power Systems, Inc.'

# Merge the two DFs on ticker

In [4]:
df_gics_w_marketcaps = df_gics.merge(df_ticker_marketcap,on="Ticker")
df_gics_w_marketcaps.drop(columns=["Unnamed: 0","CompanyName_y","Weird Sector"],inplace=True)
df_gics_w_marketcaps.rename(columns={"CompanyName_x":"CompanyName"},inplace=True)
df_gics_w_marketcaps

,Ticker,CompanyName,Sector,SubIndustry,HQLocation,DateFirstAdded,CIK,FoundedYear,QuoteURL,WikiPage,...,SectorId,Sector.1,IndustryGroupId,IndustryGroup,IndustryId,Industry,SubIndustryId,SubIndustry.1,SubIndustryDescription,MarketCap
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902,https://www.nyse.com/quote/XNYS:MMM,/wiki/3M,...,20,Industrials,2010,Capital Goods,201050,Industrial Conglomerates,20105010,Industrial Conglomerates,Diversified industrial companies with business...,73949201830
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,https://www.nyse.com/quote/XNYS:ABT,/wiki/Abbott_Laboratories,...,35,Health Care,3510,Health Care Equipment & Services,351010,Health Care Equipment & Supplies,35101010,Health Care Equipment,Manufacturers of health care equipment and dev...,183096030026
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),https://www.nyse.com/quote/XNYS:ABBV,/wiki/AbbVie_Inc.,...,35,Health Care,3520,"Pharmaceuticals, Biotechnology & Life Sciences",352020,Pharmaceuticals,35202010,Pharmaceuticals,"Companies engaged in the research, development...",253403587167
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,http://www.nasdaq.com/symbol/abmd,/wiki/Abiomed,...,35,Health Care,3510,Health Care Equipment & Services,351010,Health Care Equipment & Supplies,35101010,Health Care Equipment,Manufacturers of health care equipment and dev...,10538938584
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,https://www.nyse.com/quote/XNYS:ACN,/wiki/Accenture_plc,...,45,Information Technology,4510,Software & Services,451020,IT Services,45102010,IT Consulting & Other Services,Providers of information technology and system...,179879484123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997,https://www.nyse.com/quote/XNYS:YUM,/wiki/Yum!_Brands_Inc,...,25,Consumer Discretionary,2530,Consumer Services,253010,"Hotels, Restaurants & Leisure",25301040,Restaurants,"Owners and operators of restaurants, bars, pub...",31653183838
470,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969,http://www.nasdaq.com/symbol/zbra,/wiki/Zebra_Technologies,...,45,Information Technology,4520,Technology Hardware & Equipment,452030,"Electronic Equipment, Instruments & Components",45203010,Electronic Equipment & Instruments,Manufacturers of electronic equipment and inst...,15560235460
471,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927,https://www.nyse.com/quote/XNYS:ZBH,/wiki/Zimmer_Biomet,...,35,Health Care,3510,Health Care Equipment & Services,351010,Health Care Equipment & Supplies,35101010,Health Care Equipment,Manufacturers of health care equipment and dev...,21517306516
472,ZION,Zions Bancorp,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873,http://www.nasdaq.com/symbol/zion,/wiki/Zions_Bancorp,...,40,Financials,4010,Banks,401010,Banks,40101015,Regional Banks,Commercial banks whose businesses are derived ...,8043203684


In [5]:
df_gics_w_marketcaps['Sector'].value_counts()

Industrials               70
Information Technology    68
Financials                62
Health Care               59
Consumer Discretionary    57
Consumer Staples          32
Real Estate               29
Utilities                 28
Materials                 27
Communication Services    22
Energy                    20
Name: Sector, dtype: int64

# Define some mixture of samples by industries that are likely to contain sustainability reports 

In [6]:
gics_sample_count_dicts = {
    "Energy": 15,
    "Industrials": 15,
    "Financials": 15,
    "Consumer Discretionary": 15,
    "Consumer Staples": 15,
    "Utilities": 10,
    "Materials": 10,
    "Information Technology": 5
}
sum(gics_sample_count_dicts.values())

100

# Create sample df with top companies by market cap, with number of samples per GICS Sector of interest defined above 

In [7]:
stratified_sample_dfs = []
for gics, df_gics in df_gics_w_marketcaps.groupby("Sector"):
    if gics in gics_sample_count_dicts.keys():
        sample_size = gics_sample_count_dicts[gics]
        print(gics,f"{sample_size}")
        stratified_sample_dfs.append(df_gics.sort_values(by="MarketCap",ascending=False)[:sample_size])
df_gics_stratified = pd.concat(stratified_sample_dfs)

Consumer Discretionary 15
Consumer Staples 15
Energy 15
Financials 15
Industrials 15
Information Technology 5
Materials 10
Utilities 10


In [8]:
df_gics_stratified[df_gics_stratified["Sector"]=="Information Technology"]

,Ticker,CompanyName,Sector,SubIndustry,HQLocation,DateFirstAdded,CIK,FoundedYear,QuoteURL,WikiPage,...,SectorId,Sector.1,IndustryGroupId,IndustryGroup,IndustryId,Industry,SubIndustryId,SubIndustry.1,SubIndustryDescription,MarketCap
44,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977,http://www.nasdaq.com/symbol/aapl,/wiki/Apple_Inc.,...,45,Information Technology,4520,Technology Hardware & Equipment,452020,"Technology Hardware, Storage & Peripherals",45202030,"Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com...",2199710059533
299,MSFT,Microsoft Corp.,Information Technology,Systems Software,"Redmond, Washington",1994-06-01,789019,1975,http://www.nasdaq.com/symbol/msft,/wiki/Microsoft_Corp.,...,45,Information Technology,4510,Software & Services,451030,Software,45103020,Systems Software,Companies engaged in developing and producing ...,1898477741492
328,NVDA,Nvidia Corporation,Information Technology,Semiconductors,"Santa Clara, California",2001-11-30,1045810,1993,http://www.nasdaq.com/symbol/nvda,/wiki/Nvidia_Corporation,...,45,Information Technology,4530,Semiconductors & Semiconductor Equipment,453010,Semiconductors & Semiconductor Equipment,45301020,Semiconductors,Manufacturers of semiconductors and related pr...,421768258105
447,V,Visa Inc.,Information Technology,Data Processing & Outsourced Services,"San Francisco, California",2009-12-21,1403161,1958,https://www.nyse.com/quote/XNYS:V,/wiki/Visa_Inc.,...,45,Information Technology,4510,Software & Services,451020,IT Services,45102020,Data Processing & Outsourced Services,Providers of commercial electronic data proces...,406800625313
288,MA,Mastercard Inc.,Information Technology,Data Processing & Outsourced Services,"Harrison, New York",2008-07-18,1141391,1966,https://www.nyse.com/quote/XNYS:MA,/wiki/Mastercard_Inc.,...,45,Information Technology,4510,Software & Services,451020,IT Services,45102020,Data Processing & Outsourced Services,Providers of commercial electronic data proces...,309816525543


In [ ]:
df_gics_stratified.to_csv(DATA_DIR+"sp500-gics-stratified-sample.csv",index=False)

In [ ]:
pd.read_csv(DATA_DIR+"sp500-gics-stratified-sample.csv")